In [7]:
import pandas as pd

# Đọc file Excel
input_file = r"C:\code\Windsurf_AI\TDN\TDN_project\data\interim\sanluong_chuanhoa_ver2.xlsx"
# Đọc file Excel với engine='openpyxl'
df = pd.read_excel(input_file, engine='openpyxl')

# Lưu thành file Parquet
output_file = r"C:\code\Windsurf_AI\TDN\TDN_project\data\interim\sanluong_chuanhoa_ver2.parquet"

# Lưu file parquet
df.to_parquet(output_file, engine='pyarrow')

print(f"Đã chuyển đổi thành công từ {input_file} sang {output_file}")

Đã chuyển đổi thành công từ C:\code\Windsurf_AI\TDN\TDN_project\data\interim\sanluong_chuanhoa_ver2.xlsx sang C:\code\Windsurf_AI\TDN\TDN_project\data\interim\sanluong_chuanhoa_ver2.parquet


In [1]:
import pandas as pd

# Đọc file DGMS_BCSVH
dgms_path = r"C:\code\Windsurf_AI\TDN\TDN_project\data\interim\DGMS_BCSVH_ver2.parquet"
df_dgms = pd.read_parquet(dgms_path)

# Đọc file Sanluong
sanluong_path = r"C:\code\Windsurf_AI\TDN\TDN_project\data\interim\sanluong_chuanhoa_ver2.parquet"
df_sanluong = pd.read_parquet(sanluong_path)

print("Đã đọc thành công 2 file:")
print(f"- DGMS_BCSVH: {dgms_path}")
print(f"- Sanluong: {sanluong_path}")

Đã đọc thành công 2 file:
- DGMS_BCSVH: C:\code\Windsurf_AI\TDN\TDN_project\data\interim\DGMS_BCSVH_ver2.parquet
- Sanluong: C:\code\Windsurf_AI\TDN\TDN_project\data\interim\sanluong_chuanhoa_ver2.parquet


In [2]:
df_dgms

,Nhà máy,TD_THAMCHIEU,P.Đặt,Q.ĐK,Điện lực,Ngày,00:30,01:00,01:30,02:00,...,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30,00:00
0,Nặm Cắt,TUYEN_QUANG,3.2,PC_BACKAN,PC_BACKAN,2024.04.19,0.0,0.0,0.0,0.0,...,3.10,2.60,0.000,0.000,0.000,0.0,0.0,0.00,0.0,0.0
1,Nặm Cắt,TUYEN_QUANG,3.2,PC_BACKAN,PC_BACKAN,2024.04.20,0.0,0.0,0.0,0.0,...,3.20,3.20,3.200,3.100,2.600,1.4,1.0,0.00,0.0,0.0
2,Nặm Cắt,TUYEN_QUANG,3.2,PC_BACKAN,PC_BACKAN,2024.04.21,0.0,0.0,0.0,0.0,...,1.39,1.54,1.810,2.350,2.110,0.6,0.0,0.00,0.0,0.0
3,Nặm Cắt,TUYEN_QUANG,3.2,PC_BACKAN,PC_BACKAN,2024.04.22,0.0,0.0,0.0,0.0,...,3.20,3.20,3.055,2.715,2.275,1.9,1.6,1.42,0.0,0.0
4,Nặm Cắt,TUYEN_QUANG,3.2,PC_BACKAN,PC_BACKAN,2024.04.23,0.0,0.0,0.0,0.0,...,3.20,3.20,2.400,2.100,1.600,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99149,Đồng Sung,BAN_CHAT,20.0,A1,PC_YENBAI,2025.02.25,0.0,0.0,0.0,0.0,...,16.00,0.00,0.000,0.000,0.000,0.0,0.0,0.00,0.0,0.0
99150,Đồng Sung,BAN_CHAT,20.0,A1,PC_YENBAI,2025.02.26,0.0,0.0,0.0,0.0,...,15.00,15.00,0.000,0.000,0.000,0.0,0.0,0.00,0.0,0.0
99151,Đồng Sung,BAN_CHAT,20.0,A1,PC_YENBAI,2025.02.27,0.0,0.0,0.0,0.0,...,20.00,0.00,0.000,0.000,0.000,0.0,0.0,0.00,0.0,0.0
99152,Đồng Sung,BAN_CHAT,20.0,A1,PC_YENBAI,2025.02.28,0.0,0.0,0.0,0.0,...,21.90,0.00,0.000,0.000,0.000,0.0,0.0,0.00,0.0,0.0


In [ ]:
df_dgms['SL'] = df_dgms.loc[:, '00:30':'00:00'].sum(axis=1) / 2
df_dgms.loc[:, '00:30':'SL'] *= 1000

In [8]:
df_dgms

,Nhà máy,TD_THAMCHIEU,P.Đặt,Q.ĐK,Điện lực,Ngày,00:30,01:00,01:30,02:00,...,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30,00:00,SL
0,Nặm Cắt,TUYEN_QUANG,3.2,PC_BACKAN,PC_BACKAN,2024.04.19,0.0,0.0,0.0,0.0,...,2600.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16050.0
1,Nặm Cắt,TUYEN_QUANG,3.2,PC_BACKAN,PC_BACKAN,2024.04.20,0.0,0.0,0.0,0.0,...,3200.0,3200.0,3100.0,2600.0,1400.0,1000.0,0.0,0.0,0.0,37000.0
2,Nặm Cắt,TUYEN_QUANG,3.2,PC_BACKAN,PC_BACKAN,2024.04.21,0.0,0.0,0.0,0.0,...,1540.0,1810.0,2350.0,2110.0,600.0,0.0,0.0,0.0,0.0,5565.0
3,Nặm Cắt,TUYEN_QUANG,3.2,PC_BACKAN,PC_BACKAN,2024.04.22,0.0,0.0,0.0,0.0,...,3200.0,3055.0,2715.0,2275.0,1900.0,1600.0,1420.0,0.0,0.0,39122.5
4,Nặm Cắt,TUYEN_QUANG,3.2,PC_BACKAN,PC_BACKAN,2024.04.23,0.0,0.0,0.0,0.0,...,3200.0,2400.0,2100.0,1600.0,0.0,0.0,0.0,0.0,0.0,25900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99149,Đồng Sung,BAN_CHAT,20.0,A1,PC_YENBAI,2025.02.25,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,91150.0
99150,Đồng Sung,BAN_CHAT,20.0,A1,PC_YENBAI,2025.02.26,0.0,0.0,0.0,0.0,...,15000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90500.0
99151,Đồng Sung,BAN_CHAT,20.0,A1,PC_YENBAI,2025.02.27,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,91000.0
99152,Đồng Sung,BAN_CHAT,20.0,A1,PC_YENBAI,2025.02.28,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,108540.0


In [3]:
# Nhóm df_dgms thành các nhóm theo các giá trị unique của TD_THAMCHIEU
grouped_df = df_dgms.groupby('TD_THAMCHIEU')
grouped_df

In [ ]:
print(grouped_df.groups.keys())

dict_keys(['BAC_HA', 'BAN_CHAT', 'BAN_VE', 'HOA_BINH', 'HUA_NA/CUA_DAT', 'LAI_CHAU', 'PHO_CU', 'SON_LA', 'TUYEN_QUANG'])
